In [ ]:
# Problem 1 import libaries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import statsmodels.api as sm


In [ ]:
# Prob 1 explore data

customer_churn = pd.read_csv('Telco-Customer-Churn.csv')
print(customer_churn.head())
# Display summary statistics
print(customer_churn.describe())

In [ ]:
# Prob 1 Clean data
def load_data():
    # Load the data
    telco = pd.read_csv("Telco-Customer-Churn.csv")
    
    # Drop unnecessary columns
    telco.drop(columns=['customerID', 'PaymentMethod'], inplace=True)

    # Filter rows
    telco = telco[(telco['InternetService'] != "No") & (telco['PhoneService'] != "No")]

    # Reset index to clean up levels after filtering
    telco.reset_index(drop=True, inplace=True)

    # Drop additional columns
    telco.drop(columns=['PhoneService', 'TotalCharges'], inplace=True)

    telco['SeniorCitizen'] = telco['SeniorCitizen'].apply(lambda x: 'True' if x == 1 else 'False')

    # Convert object columns to categorical
    for col in telco.select_dtypes(include='object').columns:
        telco[col] = telco[col].astype('category')

    # Perform one-hot encoding, dropping the first category in each variable
    telco_encoded = pd.get_dummies(telco, drop_first=True)

    return telco_encoded

# Load the data
telco = load_data()

# Display the first few rows of the dataset
print(telco.head())

# Display summary statistics
print(telco.describe())

In [ ]:
# Prob 1a
# Filter rows where 'Churn' is 'Yes'
Churned = telco[telco['Churn_Yes'] == True]

# Calculate the average monthly charges for the resulting 'Churned' DataFrame
average_monthly_charges = Churned['MonthlyCharges'].mean()

# Print the average monthly charges
print(f"Average Monthly Charges for Churned Customers: ${round(average_monthly_charges,2)}")

# Assume a 12.5% profit margin
profit_margin = 0.125

# Calculate the expected cost of losing one customer (Cost 1)
cost_1 = average_monthly_charges * profit_margin

# Print the expected cost of losing one customer
print(f"Expected Cost of Losing One Customer (Cost 1): ${round(cost_1,2)}")


In [ ]:
#prob 1b
current_customer = telco[telco['Churn_Yes'] == False]

# Calculate the average monthly charges for the resulting 'current customers' DataFrame
current_customer_average_monthly_charges = current_customer['MonthlyCharges'].mean()

# Print the average monthly charges
print(f"Average Monthly Charges for current Customers: ${round(current_customer_average_monthly_charges,2)}")

# Assume a 12.5% profit margin
profit_margin_retained_customers = 0.075

# Calculate the expected cost of losing one customer (Cost 1)
cost_2 = current_customer_average_monthly_charges * profit_margin_retained_customers

# Print the expected cost of losing one customer
print(f"Expected Cost of Losing One Customer (Cost 2): ${round(cost_2,2)}")

In [28]:
# Prob 1c

# Convert boolean columns to integers (0 and 1)
bool_columns = telco.select_dtypes(include=['bool']).columns
telco[bool_columns] = telco[bool_columns].astype(int)

# Define predictor variables and response variable
X = telco.drop(columns=['Churn_Yes'])  # Predictor variables (all columns except 'Churn')
y = telco['Churn_Yes']  # Response variable

# Add a constant to the predictor variables (intercept term)
X_with_intercept = sm.add_constant(X)

# Fit the logistic regression model
logit_model = sm.GLM(y, X_with_intercept, family=sm.families.Binomial()).fit()

# Print the summary of the model
print(logit_model.summary())

ypred = (logit_model.predict(X_with_intercept) > 0.5)

# Calculate false negatives (FN) and false positives (FP)
FN = ((y == True) & (ypred == False)).sum()
FP = ((y == False) & (ypred == True)).sum()

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y, ypred)

# Create a DataFrame for better visualization
conf_matrix_df = pd.DataFrame(conf_matrix, index=['True Neg-', 'True Pos+'], columns=['Pred Neg-', 'Pred Pos+'])

# Print the confusion matrix
print(conf_matrix_df)

# Loss=Cost 1∗FN+Cost 2∗FP
# I disagree with Cost 2. It should be 5%, since that is the lost revenue due to the unecessary discount. 7.5% is the profit margin of the retain customer.

                 Generalized Linear Model Regression Results                  
Dep. Variable:              Churn_Yes   No. Observations:                 4835
Model:                            GLM   Df Residuals:                     4817
Model Family:                Binomial   Df Model:                           17
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2328.9
Date:                Thu, 21 Nov 2024   Deviance:                       4657.8
Time:                        22:46:58   Pearson chi2:                 5.08e+03
No. Iterations:                     6   Pseudo R-squ. (CS):             0.2608
Covariance Type:            nonrobust                                         
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const             

In [ ]:
# Prob 1d
def calculate_loss(y_pred_i, y):
    FN = ((y == True) & (y_pred_i == False)).sum()
    FP = ((y == False) & (y_pred_i == True)).sum()
    return cost_1*FN + cost_2*FP

# Calculate loss for 1000 evenly distributed thresholds
thresholds = np.arange(0.0, 1.0, .001)
losses = []

for threshold in thresholds:
    y_pred_i = (logit_model.predict(X_with_intercept) > threshold)
    loss = calculate_loss(y_pred_i, y)
    losses.append(loss)

print(losses)
# Find the optimal threshold
optimal_threshold = thresholds[np.argmin(losses)]
print(f"Optimal Threshold: {optimal_threshold}")

# Plot the loss with respect to the threshold
plt.figure(figsize=(10, 6))
plt.plot(thresholds, losses, marker='o', markersize=2)
plt.xlabel('Threshold')
plt.ylabel('Loss (FN + FP)')
plt.title('Loss with Respect to Threshold')
plt.grid(True)
plt.show()

In [ ]:
# Problem 4 import libaries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve



In [ ]:
# Prob 4 explore data

letters = pd.read_csv('letters-2.csv')
print(letters.dtypes)
print(letters.head())

# Display summary statistics
print(letters.describe())

In [ ]:
# 4 Prepar X and Y
# Split data
letters_shuffled = letters.copy().sample(frac=1, random_state=42).reset_index(drop=True)
train_size = int(0.75 * len(letters_shuffled))
train_set = letters_shuffled[:train_size]
test_set = letters_shuffled[train_size:]

# Display the sizes of the training and testing sets
print(f"Training set size: {train_set.shape[0]}")
print(f"Testing set size: {test_set.shape[0]}")

# Define predictor variables and response variable
X_train = train_set.drop(columns=['lettr'])  # Predictor variables (all columns except 'lettr')
y_train = train_set['lettr']  # Response variable
y_train_numeric = y_train.astype('category').cat.codes  # Encode categorical target

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled_with_intercept = sm.add_constant(X_train_scaled)

In [ ]:
# 4A with Stats Model
try:
    mnlogit_model = sm.MNLogit(y_train_numeric, X_train_scaled_with_intercept).fit()
    # Display the model summary
    print(mnlogit_model.summary())
except Exception as e:
    print(f"An error occurred while fitting the model: {e}")


In [ ]:
# 4a - SK learn fit model
sk_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42)
sk_model.fit(X_train_scaled, y_train)

coefficients = sk_model.coef_  # Shape: (n_classes, n_features)
intercepts = sk_model.intercept_  # Shape: (n_classes,)

# Classes and shape details
print(f"Classes: {sk_model.classes_}")
print(f"Number of features: {X_train_scaled.shape[1]}")
print(f"Number of classes: {len(sk_model.classes_)}")

# Coefficients
print("Coefficients (one row per class):")
print(coefficients)

# Intercepts
print("Intercepts (one per class):")
print(intercepts)


In [ ]:
#4b
# Predict the letters for the training set
train_pred = sk_model.predict(X_train_scaled)

# Display the first few predictions
print(train_pred[:10])

# Create the confusion matrix
conf_matrix = confusion_matrix(y_train, train_pred)

# Create a DataFrame for better visualization
conf_matrix_df = pd.DataFrame(conf_matrix, index=sk_model.classes_, columns=sk_model.classes_)

print(conf_matrix_df)

# Calculate classification accuracy by summing the diagonal elements and dividing by the total number of samples
correct_classifications = np.trace(conf_matrix)  # Sum of diagonal elements
total_classifications = np.sum(conf_matrix)  # Total number of samples
accuracy_manual = correct_classifications / total_classifications

print(f"Classification Accuracy (manual calculation): {accuracy_manual:.4f}")

# Calculate classification accuracy using sklearn's accuracy_score for comparison
accuracy_sklearn = accuracy_score(y_train, train_pred)
print(f"Classification Accuracy (sklearn): {accuracy_sklearn:.4f}")

In [ ]:
# 4c
# Define predictor variables and response variable
X_test = test_set.drop(columns=['lettr'])  # Predictor variables (all columns except 'lettr')
y_test = test_set['lettr']  # Response variable

scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)
# Predict the letters for the training set
test_pred = sk_model.predict(X_test_scaled)

# Display the first few predictions
print(test_pred[:10])

# Calculate classification accuracy using sklearn's accuracy_score for comparison
accuracy_sklearn = accuracy_score(y_test, test_pred)
print(f"Classification Accuracy (sklearn) test: {accuracy_sklearn:.4f}")

In [ ]:
# 4d

# Prepare the full dataset
full_set = letters.copy()

# Display the sizes of the training and testing sets
print(f"Training set size: {full_set.shape[0]}")
print(f"Testing set size: {full_set.shape[0]}")

# Define predictor variables and response variable
X = full_set.drop(columns=['lettr'])  # Predictor variables (all columns except 'lettr')
y = full_set['lettr']  # Response variable

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train a logistic regression model (multinomial)
model_full = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42)
model_full.fit(X_scaled, y)

# Read data and predict
top_secret = pd.read_csv('topsecret-1.csv')

# Ensure the top_secret dataset has the same columns and preprocess it
X_top_secret = top_secret[X.columns]  # Ensure the same columns as X
X_top_secret_scaled = scaler.transform(X_top_secret)  # Use the same scaler as training

# Predict on the top_secret dataset
pred = model_full.predict(X_top_secret_scaled)

# Display predictions
print("Predictions for top_secret dataset:")
print(pred)